In [1]:
import mne 
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

### Load Data

In [2]:
raw = mne.io.read_raw_eeglab('rEEG/sub-001/eeg/sub-001_task-Rest_eeg.set',
                              preload=True)
raw.info  # check channels, sampling rate, etc.
print(raw.ch_names)


Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-001\eeg\sub-001_task-Rest_eeg.fdt
Reading 0 ... 140829  =      0.000 ...   281.658 secs...
['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']


C:\Users\User\AppData\Local\Temp\ipykernel_46116\1015455110.py:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('rEEG/sub-001/eeg/sub-001_task-Rest_eeg.set',


### Mark Bad Channels (after visual inspection)

In [3]:
# After visual inspection, mark bad channels
raw.info['bads'] = [raw.ch_names[45], raw.ch_names[4], raw.ch_names[47]]
raw.plot(scalings='auto', block=True, title='Raw Data with Bad Channels Marked')

Using matplotlib as 2D backend.
Channels marked as bad:
['PO3', 'FT9', 'PO4']


<MNEBrowseFigure size 1920x991 with 4 Axes>

### Set montage


In [4]:
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

<RawEEGLAB | sub-001_task-Rest_eeg.fdt, 63 x 140830 (281.7 s), ~67.8 MiB, data loaded>

### Re-reference to common average

In [5]:
raw.set_eeg_reference('average')
raw.plot(scalings='auto', block=True, title='Rereferenced Data')

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Channels marked as bad:
['PO3', 'FT9', 'PO4', np.str_('T7')]


<MNEBrowseFigure size 1920x991 with 4 Axes>

### Band-pass filter (1-45 Hz)

In [6]:
raw.filter(1., 45., fir_design='firwin')
raw.plot(scalings='auto', block=True, title='Band Pass Filtered Data')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Channels marked as bad:
['PO3', 'FT9', 'PO4', np.str_('T7')]


<MNEBrowseFigure size 1920x991 with 4 Axes>

### Downsample to 250 Hz

In [7]:

raw.resample(250, npad="auto")
raw.plot(scalings='auto', block=True, title='Resampled Data')

Channels marked as bad:
['PO3', 'FT9', 'PO4', np.str_('T7')]


<MNEBrowseFigure size 1920x991 with 4 Axes>

### Plot PSD and cleaned signal

In [13]:
raw.plot(scalings='auto', block=True, title='Preprocessed Data Before ICA')
raw.plot_psd(fmax=60)

Channels marked as bad:
['PO3', 'FT9', 'PO4', np.str_('T7')]
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 8.192 (s)
Plotting power spectral density (dB=True).


<MNELineFigure size 1000x350 with 2 Axes>

## ICA (Independent Component Analysis) & Artifact Removal

In [9]:
from mne.preprocessing import ICA

ica = ICA(n_components= 20, random_state=97, max_iter='auto')
ica.fit(raw)

ica.plot_components()



Fitting ICA to data using 59 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 1.8s.


<MNEFigure size 975x967 with 20 Axes>

In [10]:
# show component topographies
ica.plot_components(picks=[0,3,6,13])

# for each candidate, check properties: time series + PSD + sources
for comp in [0,3,6,13]:
    ica.plot_properties(raw, picks=comp)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
140 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
140 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
140 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
140 matching events found
No baseline correction applied
0 projection items activated


In [11]:
ica.exclude = [0, 6]
raw_clean = ica.apply(raw.copy())



Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 2 ICA components
    Projecting back using 59 PCA components


In [12]:
raw_clean.plot(scalings = 'auto',title='After ICA', block=True)
raw_clean.plot_psd(fmax=60)

Channels marked as bad:
['PO3', 'FT9', 'PO4', np.str_('T7')]
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 8.192 (s)
Plotting power spectral density (dB=True).


<MNELineFigure size 1000x350 with 2 Axes>

### Interpolate bad channels